# 5. Global Sensitivity Analysis

This code contains a global sensitivity analysis for the policies identified in section 4- MORDM 2 of this repository

## 5.1 Installing EMA workbench and importing libraries

In [1]:
from ema_workbench import (
    Model,
    Policy,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator,
    perform_experiments,
    Samplers,
    Scenario
)
import numpy as np
import pandas as pd
import copy
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time


dike_model, planning_steps = get_model_for_problem_formulation(2)

uncertainties = copy.deepcopy(dike_model.uncertainties)

levers = copy.deepcopy(dike_model.levers)

outcome_names = []
for outcome in dike_model.outcomes:
    outcome_names.append(outcome.name)

print(outcome_names)

['Expected Annual Damage', 'Dike Investment Costs', 'RfR Investment Costs', 'Evacuation Costs', 'Expected Number of Deaths', 'Min Water Level', 'Max Water Level']


## 5.2 Defining policy and applying GSA 
The analysed policy will be 1412

In [2]:
policies_to_evaluate = [
 Policy('1412',**{'0_RfR 0': 0,
                  '0_RfR 1': 0,
                  '0_RfR 2': 0,
                  '1_RfR 0': 0,
                  '1_RfR 1': 0,
                  '1_RfR 2': 0,
                  '2_RfR 0': 0,
                  '2_RfR 1': 1,
                  '2_RfR 2': 0,
                  '3_RfR 0': 0,
                  '3_RfR 1': 0,
                  '3_RfR 2': 0,
                  '4_RfR 0': 0,
                  '4_RfR 1': 0,
                  '4_RfR 2': 0,
                  'EWS_DaysToThreat': 2,
                  'A.1_DikeIncrease 0': 5,
                  'A.1_DikeIncrease 1': 0,
                  'A.1_DikeIncrease 2': 0,
                  'A.2_DikeIncrease 0': 2,
                  'A.2_DikeIncrease 1': 0,
                  'A.2_DikeIncrease 2': 0,
                  'A.3_DikeIncrease 0': 2,
                  'A.3_DikeIncrease 1': 0,
                  'A.3_DikeIncrease 2': 0,
                  'A.4_DikeIncrease 0': 2,
                  'A.4_DikeIncrease 1': 0,
                  'A.4_DikeIncrease 2': 1,
                  'A.5_DikeIncrease 0': 3,
                  'A.5_DikeIncrease 1': 0,
                  'A.5_DikeIncrease 2': 0})
]



policy
1412    0.263193
1662    0.419249
1380    0.431670
911     0.431670

In [ ]:
scenarios = 1000

with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(scenarios, policies_to_evaluate, uncertainty_sampling=Samplers.SOBOL)



experiments, outcomes = results

outcomes_df= pd.DataFrame.from_dict(outcomes)
experiments_df= pd.DataFrame.from_dict(outcomes)
# safe outcomes to csv / json
outcomes_df.to_csv('GSA_final_outcomes')
experiments_df.to_csv('GSA_final_experiments')

C:\Users\Federico\Anaconda3\envs\MBDM\Lib\site-packages\scipy\stats\_qmc.py:763: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  sample = self._random(n, workers=workers)
  1%|▎                                   | 309/40000 [01:34<3:36:18,  3.06it/s]

In [ ]:
from ema_workbench.em_framework.salib_samplers import get_SALib_problem
from SALib.analyze import sobol

problem = get_SALib_problem(uncertainties)

results_Si =dict()

for item in outcomes_df.columns:
    results_Si[item] = sobol.analyze(problem, outcomes[item], calc_second_order=True, print_to_console=True, parallel=True)



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

Si_filter=dict()

for item in ['Expected Annual Damage', 'Expected Number of Deaths', 'Min Water Level']:
    Si_filter[item] = {k:results_Si[item][k] for k in ['ST','ST_conf','S1','S1_conf']}
    Si_df = pd.DataFrame(Si_filter[item], index=problem['names'])

    sns.set_style('white')
    fig, ax = plt.subplots(1)

    indices = Si_df[['S1','ST']]
    err = Si_df[['S1_conf','ST_conf']]

    indices.plot.bar(yerr=err.values.T,ax=ax)
    fig.set_size_inches(8,6)
    fig.subplots_adjust(bottom=0.3)
    plt.savefig(f"Final GSA policy 1412_{item}")
    plt.ylabel(f"{item}")
    plt.show()


In these graphs, the relationship between the parameters and the expected outcomes can be visualized. As was observed previously, dike rings A1 and A3 should be prioritized in a real-world policy scenario, as these are shown to have the most impact on water level. 
While A1_pfail is more strongly related to Expected Annual Damage, A3_pfail has a stronger relationship to Expected Number of Deaths. This is important information which would be useful to set priorities in a real-life scenario. 
Finally, it can be observed that the most relevant parameter for water level is flood wave shape. In reality, the usefulness of this information is not very clear, as there is no further actions that could be taken to counteract this 


In [ ]:
print(outcome_names)

S1 = how this specific parameter influences the 